In [104]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from datetime import date
import pandas as pd
date_today = date.today()

name = []
sales_price = []
price = []
product_url = []

timestamp = []
brand = []
product_type = []

items = ['coats-jackets', 'knitwear', 'fleecewear', 'overshirts', 'shirts',
       'polo-shirts-t-shirts', 'pants-5-pockets']

SCROLL_PAUSE_TIME = 3

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')

for item in items:
    
    driver = webdriver.Chrome(options=options, executable_path=r'/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
    URL = 'https://www.stoneisland.com/us/stone-island/{}'.format(item)
    time.sleep(SCROLL_PAUSE_TIME)
    driver.get(URL)

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source,'html.parser')

    for i in soup.find_all('li',{'class':'item'}):
        brand.append('Stone Island')
        timestamp.append(date_today)
        product_type.append(None)
    
        name.append(i.find('div',{'class':'info'}).find('span',{'itemprop':'name'}).text.strip())
        sales_price.append(i.find('div',{'class':'itemPrice'}).find('span',{'class':'value'}).get('data-ytos-price'))
        price.append(None)
    
        product_url.append(i.find('a',{'class':'itemLink'}).get('href'))

In [105]:
product_description = []
material = []
picture_name = []
picture_url = []
color = []
counter = 1

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')

driver = webdriver.Chrome(options=options, executable_path=r'/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
for item in product_url:
    driver.get(item)
    time.sleep(3)
    driver.find_element_by_xpath("/html/body/main/div/div[2]/div/div/div[2]/a/span[2]").click()
    time.sleep(4)
    #driver.find_element_by_xpath('/html/body/main/div/div[3]/div[1]/section/div/ul[2]/li[1]/h2/button').click()
    #time.sleep(1)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    picture_name.append('stone-island_' + str(counter) + '.jpg')
    counter += 1
    
    picture_url.append(soup.find('div',{'class':'mainImage'}).img.get('src'))
    color.append(soup.find('span',{'data-ytos-color-label':'Color'}).find('span',{'class':'text'}).text)
    
    des_index_start = str(soup.find('div',{'class':'complementary--wrapper'})).find('</div>')
    des_index = str(soup.find('div',{'class':'complementary--wrapper'})).find('<ul aria-label')
    product_description.append(str(soup.find('div',{'class':'complementary--wrapper'}))[des_index_start+6:des_index].strip())
    
    if soup.find('div',{'class':'compositionDesc'}) != None:
        material.append(soup.find('div',{'class':'compositionDesc'}).text.replace('Primary material:','').strip())
    else:
        material.append(soup.find('p',{'class':'complementary__description'}).text.strip())

In [106]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", "picture_name"]
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,Stone Island,None,00195 HAND SPRAYED OVER PRINTED SHEEPSKIN,HAND SPRAYED OVER PRINTED SHEEPSKIN: Hooded ja...,None,5133,Dove Gray,100% Sheepskin,https://www.stoneisland.com/us/stone-island/le...,https://cdn.yoox.biz/41/41969689WE_13_f.jpg,2020-12-11,stone-island_1.jpg
1,Stone Island,None,40626 MICRO REPS DOWN,MICRO REPS DOWN: Parka in a Military Specifica...,None,1265,Black,External fabric: 52% Polyester 48% Polyamide /...,https://www.stoneisland.com/us/stone-island/mi...,https://cdn.yoox.biz/41/41969725XP_13_f.jpg,2020-12-11,stone-island_2.jpg
2,Stone Island,None,40826 MICRO REPS DOWN,MICRO REPS DOWN: Hooded blouson in a Military ...,None,1020,Magenta,External fabric: 52% Polyester 48% Polyamide /...,https://www.stoneisland.com/us/stone-island/ja...,https://cdn.yoox.biz/41/41969572RP_13_f.jpg,2020-12-11,stone-island_3.jpg
3,Stone Island,None,G0426 MICRO REPS DOWN,MICRO REPS DOWN: Vest in a Military Specificat...,None,665,Blue,External fabric: 52% Polyester 48% Polyamide /...,https://www.stoneisland.com/us/stone-island/ve...,https://cdn.yoox.biz/41/41969708OD_13_f.jpg,2020-12-11,stone-island_4.jpg
4,Stone Island,None,43732 NASLAN LIGHT WATRO WITH PRIMALOFT®-TC,NASLAN LIGHT WATRO WITH PRIMALOFT®-TC: Light h...,None,1093,Color,"100% Polyamide, Wool",https://www.stoneisland.com/us/stone-island/ja...,https://cdn.yoox.biz/41/41969628HU_13_f.jpg,2020-12-11,stone-island_5.jpg


In [107]:
second_pass.to_csv("stone-island.csv", index=False, encoding = 'utf-8-sig')

In [108]:
import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/stone-island.csv', encoding='utf-8-sig')

os.mkdir('stone-island')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('stone-island/'+ "stone-island_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1